In [1]:
from rasterstats import zonal_stats
from multiprocessing import Pool
import pandas as pd
import numpy as np
import rasterio
import geopandas as gpd

### mask out the non-arable area

In [2]:
# remove the non-arable pixels
from eumap.misc import find_files, nan_percentile, ttprint
from eumap.raster import read_rasters, save_rasters
from eumap.parallel import TilingProcessing

src = ['http://192.168.1.30:8333/ai4sh/ndti.min.slopes/ndti.min.slopes_glad.landsat.ard2.seasconv.yearly.min.theilslopes_m_30m_s_20000101_20221231_eu_epsg.3035_v20231218.tif',
       'http://192.168.1.30:8333/ai4sh/trend/ndwi_glad.landsat.ard2.seasconv.yearly.m.theilslopes_m_30m_s_20000101_20221231_eu_epsg.3035_v20231218.tif',
       'http://192.168.1.30:8333/ai4sh/trend/bs_glad.landsat.ard2.seasconv.yearly.m.theilslopes_m_30m_s_20000101_20221231_eu_epsg.3035_v20231218.tif',
       'http://192.168.1.30:8333/ai4sh/trend/ndvi_glad.landsat.ard2.seasconv.yearly.m.theilslopes_m_30m_s_20000101_20221231_eu_epsg.3035_v20231218.tif']
tgt = ['/mnt/inca/tillage_index/validate_data/ndti.min_cropland_trend.tif',
       '/mnt/inca/tillage_index/validate_data/ndwi_cropland_trend.tif',
       '/mnt/inca/tillage_index/validate_data/bs_cropland_trend.tif',
       '/mnt/inca/tillage_index/validate_data/ndvi_cropland_trend.tif']

# mask_file = '/mnt/inca/tillage_index/validate_data/EUROSTATS/EUCROPMAP_2018_crop.mask.tif'
# mask, _ = read_rasters(raster_files=[mask_file], n_jobs=60)

# i = 3
# data, _ = read_rasters(raster_files=[src[i]], n_jobs=60)
# data[(np.squeeze(mask)//100)!=2, :] = 255
# save_rasters(src[i], [tgt[i]], data, n_jobs=60)

### hotspot identification

In [2]:
# after several trials, found that the nuts2 code used by EUROSTAT is 2013 version
nuts = gpd.read_file('/mnt/inca/tillage_index/data/EUROSTATS/nuts2/NUTS_RG_20M_2013_3035.shp')
nuts = nuts.loc[nuts['LEVL_CODE'].isin([0])]

stats = ['count', 'mean', 'std', 'median', 'percentile_90', 'percentile_10']
tgt = ['/mnt/inca/tillage_index/data/tif_files/ndti.min_cropland_trend.tif',
       '/mnt/inca/tillage_index/data/tif_files/min.ndti_2016.tif',
       '/mnt/inca/tillage_index/data/tif_files/min.ndti_2010.tif']

# '/mnt/inca/tillage_index/data/tif_files/ndwi_cropland_trend.tif',
#        '/mnt/inca/tillage_index/data/tif_files/bs_cropland_trend.tif',
#        '/mnt/inca/tillage_index/data/tif_files/ndvi_cropland_trend.tif',

def calculate_zonal_stats(args):
    index, geometry, stats, raster_path = args
    result = {stat: np.nan for stat in stats}
    result['count_zero'] = np.nan  # Include count_zero in the initialized results

    try:
        # Calculate the standard zonal statistics
        zonal_result = zonal_stats(geometry, raster_path, stats=stats)
        if zonal_result:
            result.update(zonal_result[0])

        # Open the raster file and read the values for the specified geometry
        with rasterio.open(raster_path) as src:
            # Convert geometry into a format that rasterio can use (from GeoJSON-like dict to a shape)
            if isinstance(geometry, gpd.GeoDataFrame):
                geometry = geometry.__geo_interface__
            elif isinstance(geometry, gpd.GeoSeries):
                geometry = geometry.values[0].__geo_interface__

            # Create a mask for the geometry
            out_image, out_transform = rasterio.mask.mask(src, [geometry], crop=True)
            count_zero = (out_image == 0).sum()

            result['count_zero'] = count_zero

    except ValueError as e:
        print(f"Error for index {index}: {e}")

    return index, result

name = ['trend','2016','2010']


In [ ]:
from rasterio.mask import mask

for idd in [0, 1, 2]:
    print(tgt[idd])
    args = [(index, row.geometry, stats, tgt[idd]) for index, row in nuts.iterrows()]
    
    results = []
    for arg in args:
        result = calculate_zonal_stats(arg)
        results.append(result)
        
    sorted_results = sorted(results, key=lambda x: x[0])
    sorted_results = [result for index, result in sorted_results]
    results_df = pd.DataFrame(sorted_results, index=nuts.index)
    
    name_map = {stat: f'{name[idd]}_{stat}' for stat in stats}
    results_df = results_df.rename(columns=name_map)
    
    # The condition if idd==4 seems to be a mistake since the loop goes through indices 0, 1, 2.
    # Assuming there's no specific action required for index 4, we can remove that condition.
    # Instead, we merge the results directly into the nuts DataFrame.
    nuts = nuts.merge(results_df, left_index=True, right_index=True)


/mnt/inca/tillage_index/data/tif_files/ndti.min_cropland_trend.tif


In [ ]:
# for idd in [0,1,2]:
#     print(tgt[idd])
#     args = [(index, row.geometry, stats, tgt[idd]) for index, row in nuts.iterrows()]
    
#     with Pool(50) as pool:
#         results = pool.map(calculate_zonal_stats, args)
        
#     sorted_results = sorted(results, key=lambda x: x[0])
#     sorted_results = [result for index, result in sorted_results]
#     results_df = pd.DataFrame(sorted_results, index=nuts.index)
    
#     name_map =  {stat:f'{name[idd]}_{stat}' for stat in stats}
#     results_df = results_df.rename(columns=name_map)
#     if idd==4:
#         nuts = nuts.drop(columns=['count_zero_y','count_zero_x','count_zero'])
#         nuts = nuts.merge(results_df, left_index=True, right_index=True)
#     else:
#         nuts = nuts.merge(results_df, left_index=True, right_index=True)